# MD-cGAN y FID

Para poder ejecutar el algoritmo en ***Kaggle*** y poder hacer uso de GPU de Kaggle insertamos el siguiente código:

In [4]:
# KAGGLE
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Implementación de la red MD-cGAN

### Paquetes necesarios

Importamos los paquetes necesarios

In [ ]:
# importamos paquetes necesarios
from tensorflow import keras

from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.datasets.fashion_mnist import load_data
#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Average
from matplotlib import pyplot

### Modelo discriminador

In [ ]:
# definimos el modelo discriminador
def define_discriminator(in_shape=(28,28,1), n_classes=10):
    # etiquetas de entrada
    in_label = Input(shape=(1,))
    # incrustamos las etiquetas
    li = Embedding(n_classes, 50)(in_label)
    # escalamos al tamaño de las imágenes
    n_nodes = in_shape[0] * in_shape[1]
    li = Dense(n_nodes)(li)
    # remodelamos un canal adicional
    li = Reshape((in_shape[0], in_shape[1], 1))(li)
    # entrada de imágenes
    in_image = Input(shape=in_shape)
    # concatenamos imagen y etiqueta
    merge = Concatenate()([in_image, li])
    # reducción por convolución 28x28 -> 14x14
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
    fe = LeakyReLU(alpha=0.2)(fe)
    # reducción por convolución 14x14 -> 7x7
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # aplanado
    fe = Flatten()(fe)
    # dropout
    fe = Dropout(0.4)(fe)
    # output
    out_layer = Dense(1, activation='sigmoid')(fe)
    # definimos el modelo
    model = Model([in_image, in_label], out_layer)
    # compilamos el modelo
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

### Modelo generador

In [ ]:
# definimos el modelo generador
# proporcionamos el espacio latente y el número de clases
def define_generator(latent_dim, n_classes=10):
    # etiquetas de entrada
    in_label = Input(shape=(1,))
    # incrustamos las etiquetas
    li = Embedding(n_classes, 50)(in_label)
    # multiplicación
    n_nodes = 7 * 7
    li = Dense(n_nodes)(li)
    # remodelamos un canal adicional
    li = Reshape((7, 7, 1))(li)
    # entrada de imágenes
    in_lat = Input(shape=(latent_dim,))
    # base de la imagen 7x7
    n_nodes = 128 * 7 * 7
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((7, 7, 128))(gen)
    # concatenamos imagen y etiqueta
    merge = Concatenate()([gen, li])
    # aumentamos por convolución 7x7 -> 14x14
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)
    gen = LeakyReLU(alpha=0.2)(gen)
    # aumentamos por convolución 14x14 -> 28x28
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # output
    out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
    # definimos el modelo (no lo compilamos)
    model = Model([in_lat, in_label], out_layer)
    return model

### Modelo GAN

In [ ]:
# definimos el modelo combinado de generador-discriminador, para actualizar el generador
def define_gan(g_model, d_model, num_d):
    y = []
    # congelamos los modelos discriminadores para que no entrenen
    for n_d in range(num_d):
        d_model[n_d].trainable = False
    # obtenemos la entrada de ruido y etiquetas del modelo generador
    gen_noise, gen_label = g_model.input
    # obtenemos la imagen de salida del modelo generador
    gen_output = g_model.output
    # conectamos la imagen y la etiqueta del generador como entradas del discriminador
    #gan_output = d_model([gen_output, gen_label])
    if num_d==1:
        gan_output = d_model[0]([gen_output, gen_label])
    else:
        for n_d in range(num_d):
            y.append(d_model[n_d]([gen_output, gen_label]))
        # definimos el cálculo (Average, Maximum ó Minimum)
        gan_output = Maximum()(y)
    # definimos el modelo combinado gan
    model = Model([gen_noise, gen_label], gan_output)
    # compilamos el modelo
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

### Carga de datos

In [ ]:
# cargamos las imágenes del fashion-MNIST
def load_real_samples():
    # cargamos los datos de entrenamiento
    (trainX, trainy), (_, _) = load_data()
    # añadimos un canal -> 3D
    X = expand_dims(trainX, axis=-1)
    # convertimos de entero a flotante
    X = X.astype('float32')
    # escalamos [0, 255] -> [-1, 1]
    X = (X - 127.5) / 127.5
    return [X, trainy]

### Selección de imágenes reales

Realizamos una selección aleatoria de imágenes reales del dataset importado.

In [ ]:
# seleccionamos aleatoriamente un grupo de imágenes reales
def generate_real_samples(dataset, n_samples):
    # dividimos en imágenes y etiquetas
    images, labels = dataset
    # selección aleatoria de índices
    ix = randint(0, images.shape[0], n_samples)
    # selección aleatoria de imágenes y etiquetas
    X, labels = images[ix], labels[ix]
    # generamos las etiquetas de clase 1 -> real (0 -> fake)
    y = ones((n_samples, 1))
    return [X, labels], y

### Espacio latente

In [ ]:
# generamos puntos en el espacio latente como entrada para el generador
def generate_latent_points(latent_dim, n_samples, n_classes=10):
    # generamos aleatoriamente puntos
    x_input = randn(latent_dim * n_samples)
    # redimensionamos
    z_input = x_input.reshape(n_samples, latent_dim)
    # generamos las etiquetas
    labels = randint(0, n_classes, n_samples)
    return [z_input, labels]

### Generamos imágenes falsas (fake)

In [ ]:
# utilizamos el generador para crear imágenes falsas
# con etiquetas de clase (0 -> fake)
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generamos puntos en el espacio latente
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predicción
    images = g_model.predict([z_input, labels_input])
    # generamos las etiquetas de clase 0 -> fake (1 -> real)
    y = zeros((n_samples, 1))
    return [images, labels_input], y


### Guardamos grupo de imágenes (10x10)

In [ ]:
# guardamos un plot con las imágenes generadas (10x10)
def save_plot(examples, epoch, n=10):
    # creamos el plot
    for i in range(n * n):
        # subplot
        pyplot.subplot(n, n, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot
        pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
    # guardamos el fichero de imágenes
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
pyplot.close()

### Evaluamos el modelo discriminador

In [ ]:
# evaluamos el modelo discriminador, generamos imágenes y guardamos el modelo generador
def summarize_performance(epoch, g_model, d_model, num_d, dataset, latent_dim, n_samples=100):
    acc_real = []
    acc_fake = []
    #tot_acc_real = []
    #tot_acc_fake = []
    #tot_epoch = []
    # muestras de imágenes reales
    [X_real, labels_real], y_real = generate_real_samples(dataset, n_samples)
    # muestras de imágenes fake
    [X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluamos el discriminador con las muestras reales y falsas
    for n_d in range(num_d):
        _, acc_real.append(d_model[n_d].evaluate([X_real, labels_real], y_real, verbose=0))
        _, acc_fake.append(d_model[n_d].evaluate([X_fake, labels_fake], y_fake, verbose=0))
        # rendimiento del discriminador
        print('>Accuracy real_%d: %.0f%%, fake_%d: %.0f%%' % (n_d+1, acc_real[n_d][-1]*100, n_d+1, acc_fake[n_d][-1]*100))
        # guardamos el rendimiento del discriminador
        #tot_acc_real.append(acc_real[n_d]*100)
        #tot_acc_fake.append(acc_fake[n_d]*100)
    #tot_epoch.append(epoch +1)
    # muestras de imágenes fake
    latent_points, labels = generate_latent_points(100, 100)
    # etiquetas
    labels = asarray([x for _ in range(10) for x in range(10)])
    # generamos las imágenes fake
    X  = g_model.predict([latent_points, labels])
    # escalamos [-1, 1] -> [0, 1]
    X = (X + 1) / 2.0
    # salvamos el gráfico
    save_plot(X, epoch)
    
    # guardamos el modelo generador
    filename = 'generator_model_%03d.h5' % (epoch + 1)
    g_model.save(filename)

### Entrenamos el modelo generador y discriminador

In [ ]:
# entrenamos el modelo generador y discriminador
def train(g_model, d_model, num_d, gan_model, dataset, latent_dim, n_epochs, n_batch):
    d_loss_real = []
    d_loss_fake = []

    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # recorremos las épocas
    for i in range(n_epochs):
        # recorremos los lotes definidos por época
        for j in range(bat_per_epo):
            # muestras aleatorias de imágenes reales
            [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
            # muestras de imágenes fake
            [X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            
            
            
            # actualizamos los pesos del modelo discriminador
            for n_d in range(num_d):
                d_loss_real.append(d_model[n_d].train_on_batch([X_real, labels_real], y_real))
                d_loss_fake.append(d_model[n_d].train_on_batch([X_fake, labels_fake], y_fake))
                
                
            # preparamos puntos en el espacio latente como entrada para el modelo generador
            [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
            # creamos etiquetas falsas para las muestras fake (en lugar de 0)
            y_gan = ones((n_batch, 1))
            # actualizamos el modelo generador mediante el error del discriminador
            g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
            
            # pérdidas del lote
            for n_d in range(num_d):
                print('>%d, %d/%d, d_real_%d=%.3f, d_fake%d=%.3f g=%.3f' %
                (i+1, j+1, bat_per_epo, n_d+1, d_loss_real[n_d][-1], n_d+1, d_loss_fake[n_d][-1], g_loss))

        # evaluamos el rendimiento del modelo cada 10 épocas
        if (i+1) % 10 == 0:
            summarize_performance(i, g_model, d_model, num_d, dataset, latent_dim)

### Parametros de la GAN

In [ ]:
# épocas
n_epochs = 200
# lote
n_batch = 128
# tamaño del espacio latente (100, 10, 50, 500)
latent_dim = 100
# num discriminators
num_d = 3

# creamos los modelos discriminadores
d_model = []
for n_d in range(num_d):
     d_model.append(define_discriminator())
    
# creamos el modelo generador
g_model = define_generator(latent_dim)
# creamos el modelo combinado
gan_model = define_gan(g_model, d_model, num_d)
# cargamos imágenes reales
dataset = load_real_samples()
# entrenamos la red
train(g_model, d_model, num_d, gan_model, dataset, latent_dim, n_epochs, n_batch)

## Cálculo del FID

Una vez que tenemos los diferentes modelos obtenidos para las distintas épocas de entrenamiento tenemos que evaluar lo bien que genera imágenes ese modelo

### FID

Para llevar a cabo el cálculo del FID necesitamos evaluar las imágenes mediante un modelo para ello utilizamos el modelo ya optimizado *InceptionV3*

### Paquetes necesarios

In [ ]:
# Calculamos el FID (Frechet inception distance) para fashion_mnist
# importamos paquetes necesarios
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
from keras.datasets import cifar10
from keras.datasets import fashion_mnist

### Escalado de las imágenes

In [ ]:
# escalamos las imágenes
def scale_images(images, new_shape):
    images_list = list()
    for image in images:
        # resize with nearest neighbor interpolation
        new_image = resize(image, new_shape, 0)
        # guardamos
        images_list.append(new_image)
    return asarray(images_list)

### Calculamos el FID

In [ ]:
# cálculo del FID
def calculate_fid(model, images1, images2):
    # predicción del modelo InceptionV3
    act1 = model.predict(images1)
    act2 = model.predict(images2)
    # calculamos la media y covarianza
    mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
    # calculamos la suma de los cuadrados de la difencia de medias
    ssdiff = numpy.sum((mu1 - mu2)**2.0)
    # calculamos la raiz cuadrada del producto de las covarianzas
    covmean = sqrtm(sigma1.dot(sigma2))
    # comprobamos y corregimos los números imaginarios de la raiz
    if iscomplexobj(covmean):
        covmean = covmean.real
    # calculamos el FID
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

### Preparamos el modelo *Inception v3*

In [ ]:
# prepamos el modelo inception v3 (discriminador)
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

### Preparamos los modelos generadores

In [ ]:
# generamos imágenes con modelo generador

# importamos paquetes
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.models import load_model

# parametros
n_paquetes = 100
n_classes = 10
num_imag = n_paquetes * n_classes

# generamos espacio latente
latent_points, labels = generate_latent_points(100, num_imag, n_classes)

# etiquetas
labels = asarray([x for _ in range(n_paquetes) for x in range(n_classes)])

# cargamos el modelo
fid_epoch = []
epoch = []

### Generamos pool de imágenes reales

In [ ]:
# generamos imágenes reales
dataset = load_real_samples()
n_samples = 1000
[X_real, labels_real], y_real = generate_real_samples(dataset, n_samples)
X_real = X_real.astype('float32')

#escalamos las imágenes
X_real = scale_images(X_real, (299,299,3))

# preprocesamos imágenes
X_real = preprocess_input(X_real)

### Calculamos FID para todos los modelos

In [ ]:
for i in range(20):
    model_guardado = 'generator_model_%03d.h5' % ((i + 1) * 10)
    model_guardado = load_model(model_guardado)
    
    # generamos imágenes
    X_fake = model_guardado.predict([latent_points, labels])
    X_fake = X_fake.astype('float32')
    # escalamos las imágenes
    X_fake = scale_images(X_fake, (299,299,3))
    print('Scaled', X_real.shape, X_fake.shape)
    # preprocesamos imágenes
    X_fake = preprocess_input(X_fake)
    # calculamos el fid
    fid = calculate_fid(model, X_real, X_fake)
    fid_epoch.append(fid)
    epoch.append((i+1) * 10)
    print('FID: %.3f' % fid)

### Gráfico del FID

In [ ]:
# graficamos
import numpy as np
from matplotlib import pyplot as plt
x = epoch
y = fid_epoch

fig = plt.figure()
fig.clf()
ax = fig.add_subplot(1,1,1)
ax.clear()
ax.plot(x, y,  marker='D')
ax.set_xlabel('epoch')
ax.set_ylabel('fid')
ax.set_title('FID por épocas')
fig.show()

Podemos guardar el gráfico resultante.

In [ ]:
fig.savefig("FID.png", bbox_inches='tight')